Create a Run in an Experiment in MLFlow

In [ ]:
import os
import warnings
import sys

import pandas as pd
import numpy as np

from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn import linear_model

import mlflow
import mlflow.sklearn

Set the MLFlow-Tracking Server

In [ ]:
mlflow.set_tracking_uri("http://mlflow_server:6798")

Set Experiment to which the Run is to be added

In [ ]:
exp_id = "XXXXXXXXXXXX"

mlflow.set_experiment(experiment_id=exp_id)


Set the Metrics on which the Run should be evaluated
Possible Metrics: \n

| Description      | Class |
| ----------- | ----------- |
| Explained variance regression score function      | explained_variance_score       |
| The max_error metric calculates the maximum residual error   | max_error        |
|Mean absolute error regression loss|mean_absolute_error|
|Mean squared error regression loss|mean_squared_error|
|Mean squared logarithmic error regression loss|mean_squared_log_error|
|Median absolute error regression loss|median_absolute_error|
|Mean absolute percentage error (MAPE) regression loss|mean_absolute_percentage_error|
|R2(coefficient of determination) regression score function|r2_score|
|Mean Poisson deviance regression loss|mean_poisson_deviance|
|Mean Gamma deviance regression loss|mean_gamma_deviance|
|Mean Tweedie deviance regression loss|mean_tweedie_deviance|
|D2 regression score function, fraction of Tweedie deviance explained|d2_tweedie_score|
|Pinball loss for quantile regression|mean_pinball_loss|
|D2 regression score function, fraction of pinball loss explained|d2_pinball_score|
|D2 regression score function, fraction of absolute error explained|d2_absolute_error_score|
|Accuracy classification score|accuracy_score|
|Compute Area Under the Curve (AUC) using the trapezoidal rule|auc|
|Compute average precision (AP) from prediction scores|average_precision_score|
|Compute the balanced accuracy|balanced_accuracy_score|
|Compute the Brier score loss|brier_score_loss|
|Build a text report showing the main classification metrics|classification_report|
|Compute Cohen's kappa: a statistic that measures inter-annotator agreement|cohen_kappa_score|
|Compute confusion matrix to evaluate the accuracy of a classification|confusion_matrix|
|Compute the F1 score, also known as balanced F-score or F-measure|f1_score|
|Compute the F-beta score|fbeta_score|
|Compute the average Hamming loss|hamming_loss|
|Jaccard similarity coefficient score|jaccard_score|
|Log loss, aka logistic loss or cross-entropy loss|log_loss|
|Compute the Matthews correlation coefficient (MCC)|matthews_corrcoef|
|Compute a confusion matrix for each class or sample|multilabel_confusion_matrix|
|Compute precision-recall pairs for different probability thresholds|precision_recall_curve|
|Compute precision, recall, F-measure and support for each class|precision_recall_fscore_support|
|Compute the precision|precision_score|
|Compute the recall|recall_score|
|Zero-one classification loss|zero_one_loss|

In [ ]:
#Example with mean_squared_error, mean_absolute_error, r2_score
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [ ]:
def load_data_dataSet_json(data_path, column):
    data = pd.read_json(data_path)
    train, test = train_test_split(data)
    train_x = train.drop([column], axis=1)
    test_x = test.drop([column], axis=1)
    train_y = train[[column]]
    test_y = test[[column]]
    return train_x, train_y, test_x, test_y

Trainings-Function
Possible Model-Classes:
| Classes | Parameters |
| ----------- | ----------- |
| linear_model.LinearRegression      |*, fit_intercept=True, normalize='deprecated', copy_X=True, n_jobs=None, positive=False|
| linear_model.Ridge      | alpha=1.0, *, fit_intercept=True, normalize='deprecated', copy_X=True, max_iter=None, tol=0.001, solver='auto', positive=False, random_state=None|
|linear_model.Lasso| alpha=1.0, *, fit_intercept=True, normalize='deprecated', precompute=False, copy_X=True, max_iter=1000, tol=0.0001, warm_start=False, positive=False, random_state=None, selection='cyclic'|
|linear_model.MultiTaskLasso| alpha=1.0, *, fit_intercept=True, normalize='deprecated', copy_X=True, max_iter=1000, tol=0.0001, warm_start=False, random_state=None, selection='cyclic'|
|linear_model.ElasticNet| alpha=1.0, *, l1_ratio=0.5, fit_intercept=True, normalize='deprecated', precompute=False, max_iter=1000, copy_X=True, tol=0.0001, warm_start=False, positive=False, random_state=None, selection='cyclic'|
|linear_model.MultiTaskElasticNet| alpha=1.0, *, l1_ratio=0.5, fit_intercept=True, normalize='deprecated', copy_X=True, max_iter=1000, tol=0.0001, warm_start=False, random_state=None, selection='cyclic'|
|linear_model.Lars| *, fit_intercept=True, verbose=False, normalize='deprecated', precompute='auto', n_nonzero_coefs=500, eps=2.220446049250313e-16, copy_X=True, fit_path=True, jitter=None, random_state=None|
|linear_model.LassoLars|alpha=1.0, *, fit_intercept=True, verbose=False, normalize='deprecated', precompute='auto', max_iter=500, eps=2.220446049250313e-16, copy_X=True, fit_path=True, positive=False, jitter=None, random_state=None |
|linear_model.OrthogonalMatchingPursuit|*, n_nonzero_coefs=None, tol=None, fit_intercept=True, normalize='deprecated', precompute='auto' |
|linear_model.BayesianRidge|*, n_iter=300, tol=0.001, alpha_1=1e-06, alpha_2=1e-06, lambda_1=1e-06, lambda_2=1e-06, alpha_init=None, lambda_init=None, compute_score=False, fit_intercept=True, normalize='deprecated', copy_X=True, verbose=False |
|linear_model.LogisticRegression|penalty='l2', *, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None |
|linear_model.TweedieRegressor|*, power=0.0, alpha=1.0, fit_intercept=True, link='auto', max_iter=100, tol=0.0001, warm_start=False, verbose=0 |
|linear_model.SGDRegressor|loss='squared_error', *, penalty='l2', alpha=0.0001, l1_ratio=0.15, fit_intercept=True, max_iter=1000, tol=0.001, shuffle=True, verbose=0, epsilon=0.1, random_state=None, learning_rate='invscaling', eta0=0.01, power_t=0.25, early_stopping=False, validation_fraction=0.1, n_iter_no_change=5, warm_start=False, average=False |
|linear_model.Perceptron| *, penalty=None, alpha=0.0001, l1_ratio=0.15, fit_intercept=True, max_iter=1000, tol=0.001, shuffle=True, verbose=0, eta0=1.0, n_jobs=None, random_state=0, early_stopping=False, validation_fraction=0.1, n_iter_no_change=5, class_weight=None, warm_start=False|
|linear_model.PassiveAggressiveClassifier|*, C=1.0, fit_intercept=True, max_iter=1000, tol=0.001, early_stopping=False, validation_fraction=0.1, n_iter_no_change=5, shuffle=True, verbose=0, loss='hinge', n_jobs=None, random_state=None, warm_start=False, class_weight=None, average=False |
|linear_model.HuberRegressor|*, epsilon=1.35, max_iter=100, alpha=0.0001, warm_start=False, fit_intercept=True, tol=1e-05|
|linear_model.QuantileRegressor| *, quantile=0.5, alpha=1.0, fit_intercept=True, solver='interior-point', solver_options=None|
|linear_model.LinearRegression| *, fit_intercept=True, normalize='deprecated', copy_X=True, n_jobs=None, positive=False|


In [ ]:
def train(parameters):
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    datasets = []
    #Define the train Column
    train_x, train_y, test_x, test_y = load_data_dataSet_json(datasets[0], "XXXXXXXXX")
    with mlflow.start_run():
        # Execute Model
        # Example here with ElasticNet
        # look at other Possibilities above
        lr = linear_model.ElasticNet(alpha=parameters[0], l1_ratio=parameters[1], random_state=42)
        lr.fit(train_x, train_y)
        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        #Change to above defined Metrics 
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)
        # Log parameter, metrics, and model to MLflow
        # Change to the Parameters defined in the section above
        
        mlflow.log_param(key="XXXX", value=parameters[0])
        mlflow.log_param(key="l1_ratio", value=parameters[1])
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        
        data_path = "/home/jovyan/work/tmp/dataset.txt" 
        with open(data_path, 'w') as f:
            f.write(datasets[0])
        mlflow.log_artifact(data_path)
        os.remove(data_path)

        
        mlflow.sklearn.log_model(lr, "model")

Train the Modell

In [ ]:
train([0.5, 0.5])